# Labyrinthe Lösen
- Analogie für viele gängige Suchaufgaben in der Informatik
- Dient der Veranschaulichung
---
## Das Labyrinth erzeugen

In [1]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt
from generic_search import dfs, node_to_path, Node, bfs, astar
class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"
class MazeLocation(NamedTuple):
    row: int
    column: int
class Maze:
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2, start: MazeLocation = MazeLocation(0, 0), goal: MazeLocation = MazeLocation(9, 9)) -> None:
        # Grundlegende Instanzvariablen initialisieren
        self.rows: int = rows
        self.columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        # Das Gitter mit Zellen füllen
        self._grid: List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]
        # Gitter mit besetzten Zellen bestücken
        self._randomly_fill(rows, columns, sparseness)
        # Start und Ziel Positionen einfügen
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self.rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self.columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    def mark(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
    
if __name__ == "__main__":
    # DFS Testen
    m: Maze = Maze()
    print("Das Labyrinth:")
    print(m)

Das Labyrinth:
S     XX  
        XX
 X   XXX  
 X     X X
   X      
   X   X  
X       X 
XX X    X 
   X   XX 
X X    X G



## Tiefensuche
- Tiefensuche wird genutzt um Schritt für Schritt einen Pfad zu finden


In [2]:
solution1: Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test, m.successors) # DFS
if solution1 is None:
        print("Die Tiefensuche hat keinen Weg gefunden")
else:
        path1: List[MazeLocation] = node_to_path(solution1)
        m.mark(path1)
        print("Das Labyrinth mit den gefundenem Weg der Tiefensuche:")
        print(m)
        m.clear(path1)

Das Labyrinth mit den gefundenem Weg der Tiefensuche:
S**** XX  
    *   XX
 X***XXX  
 X*    X X
 **X  ****
 * X  *X *
X****** X*
XX X    X*
   X   XX*
X X    X G



## Breitensuche
- Die Breitsuche wird genutzt um Schicht für Schicht, den kürzesten Pfad zu ermitteln
- Diese Suche kann sehr aufwendig sein

In [3]:
solution2: Optional[Node[MazeLocation]] = bfs(m.start, m.goal_test, m.successors) # BFS
if solution2 is None:
        print("Die Breitensuche hat keinen Weg gefunden")
else:
        path2: List[MazeLocation] = node_to_path(solution2)
        m.mark(path2)
        print("Das Labyrinth mit den gefundenem Weg der Breitensuche:")
        print(m)
        m.clear(path2)

Das Labyrinth mit den gefundenem Weg der Breitensuche:
S     XX  
***     XX
 X*  XXX  
 X***  X X
   X***** 
   X   X**
X       X*
XX X    X*
   X   XX*
X X    X G



## A* Suche
- Verwendet Kosten und Heuristik Funktion
- g(n) [Kostenfunktion] gibt den Aufwand zu einem Zustand zu gelangen wieder
- h(n) [Heuristikfunktion] gibt eine Schätzung der Kosten von einem Zustand zu einem anderen zu gelangen wieder
- Die Gesamkosten ergeben sich aus der Kombination von Kosten und Heuristik Funktion
    - f(n) =  g(n) + h(n)
- Es wird die Funktion mit den niedrigsten Gesamtkosten ausgewählt

In [4]:
def euclidean_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = ml.column - goal.column
        ydist: int = ml.row - goal.row
        return sqrt((xdist * xdist) + (ydist * ydist))
    return distance

def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = abs(ml.column - goal.column)
        ydist: int = abs(ml.row - goal.row)
        return xdist + ydist
    return distance

# A* Testen
distance: Callable[[MazeLocation], float] = manhattan_distance(m.goal)
solution3: Optional[Node[MazeLocation]] = astar(m.start, m.goal_test, m.successors, distance) # A* mit Manhattan-Distanz
if solution3 is None:
        print("Die A*-Suche hat keinen Weg gefunden")
else:
        path3: List[MazeLocation] = node_to_path(solution3)
        m.mark(path3)
        print("Das Labyrinth mit den gefundenem Weg der A*-Suche:")
        print(m)
        m.clear(path3)

Das Labyrinth mit den gefundenem Weg der A*-Suche:
S     XX  
***     XX
 X***XXX  
 X  ***X X
   X  ****
   X   X *
X       X*
XX X    X*
   X   XX*
X X    X G

